In [23]:

from sklearn.neighbors import NearestNeighbors
import os
import numpy as np
import librosa

def extract_features(file_path, max_length=1000):
    audio, sr = librosa.load(file_path, mono=True)
    # 1. Coeficientes MFCC
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=100)
    mfcc_features = np.concatenate((mfccs.mean(axis=1), mfccs.std(axis=1)))

    # 2. Delta MFCC
    delta_mfccs = librosa.feature.delta(mfccs)
    delta_mfcc_features = np.concatenate((delta_mfccs.mean(axis=1), delta_mfccs.std(axis=1)))

    # 3. Delta Delta MFCC
    delta2_mfccs = librosa.feature.delta(mfccs, order=2)
    delta2_mfcc_features = np.concatenate((delta2_mfccs.mean(axis=1), delta2_mfccs.std(axis=1)))

    # Representan la información espectral de la señal de audio.
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    # 4. Contraste espectral
    contrast = librosa.feature.spectral_contrast(y=audio, sr=sr).
    # 5. Tonnetz
    tonnetz = librosa.feature.tonnetz(y=audio, sr=sr)

    # 6. Tempograma
    tempo, tempogram = librosa.beat.beat_track(y=audio, sr=sr)

    # Asegurar que todas las características tengan la misma longitud
    all_features = np.concatenate((
        mfcc_features,
        delta_mfcc_features,
        delta2_mfcc_features,
        chroma.mean(axis=1),
        contrast.mean(axis=1),
        tonnetz.mean(axis=1),
        tempogram
    ))

    # Rellenar con ceros si es necesario
    if len(all_features) < max_length:
        all_features = np.pad(all_features, (0, max_length - len(all_features)))
    else:
        # Recortar si es necesario
        all_features = all_features[:max_length]

    return all_features

In [24]:
import os
def listar_archivos_carpeta(ruta_carpeta):
    archivos = [arch for arch in os.listdir(ruta_carpeta) if arch.endswith(('.mp3'))]
    return archivos

In [25]:
import random
ruta_carpeta = r"C:\Users\diego\DataspellProjects\spotify\es"
archivos_carpeta = listar_archivos_carpeta(ruta_carpeta)
archivos_muestra = random.sample(archivos_carpeta,50)
# Almacena las características y las etiquetas (nombre de archivo) en listas
caracteristicas = []
etiquetas = []

for archivo in archivos_muestra:
    ruta_archivo = os.path.join(ruta_carpeta, archivo)
    features = extract_features(ruta_archivo)
    caracteristicas.append(features)
    etiquetas.append(archivo)

In [26]:
from sklearn.preprocessing import StandardScaler
X = np.array(caracteristicas)

# Normaliza los datos para mejorar el rendimiento del algoritmo KNN
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Crea un modelo KNN con distancia euclidiana
knn_model = NearestNeighbors(n_neighbors=3, metric='euclidean')
knn_model.fit(X_normalized)

NearestNeighbors(metric='euclidean', n_neighbors=3)

In [27]:
query_file = r'C:\Users\diego\DataspellProjects\spotify\es\Farruko - Hola Beba.mp3'  # Reemplaza con el nombre de tu archivo de 
query_features = extract_features(os.path.join(ruta_carpeta, query_file))
query_features_normalized = scaler.transform([query_features])  # Normaliza los datos de consulta

# Realiza la consulta KNN
distances, indices = knn_model.kneighbors(query_features_normalized)

# Imprime los vecinos más cercanos y sus distancias
print("Vecinos más cercanos:")
for i, index in enumerate(indices.flatten()):
    print(f"{etiquetas[index]} - Distancia: {distances.flatten()[i]}")

Vecinos más cercanos:
Farruko - Hola Beba.mp3 - Distancia: 0.0
Zion - Fantasma.mp3 - Distancia: 27.033588520606692
Don Omar, Lucenzo - Danza Kuduro.mp3 - Distancia: 28.993762322346345


In [29]:
len(caracteristicas[0])

1000

In [17]:
caracteristicas

[array([-8.10660782e+01,  8.66331024e+01, -1.30871954e+01,  2.05245323e+01,
         1.53280938e+00,  4.47197104e+00, -3.82708168e+00, -2.97394586e+00,
        -2.64148331e+00,  3.39849138e+00, -5.73201942e+00, -8.53113770e-01,
        -1.05239856e+00, -9.10729051e-01, -3.17901802e+00,  1.17447758e+00,
        -6.47873068e+00, -2.05823922e+00, -5.47078466e+00, -2.46461391e+00,
         9.09373474e+01,  3.24855843e+01,  2.53652992e+01,  1.90167408e+01,
         1.66255779e+01,  1.43220978e+01,  1.44523754e+01,  1.29133558e+01,
         1.26778831e+01,  1.26355667e+01,  1.27167006e+01,  1.08674068e+01,
         1.20322123e+01,  1.07309523e+01,  9.20154381e+00,  9.90784836e+00,
         9.51732349e+00,  8.64599991e+00,  9.27200127e+00,  8.98191833e+00,
         4.05118517e-09, -3.03838887e-09,  4.30438440e-09, -2.53199073e-10,
        -1.01279629e-09,  1.13939580e-09,  1.26599542e-09, -5.06398146e-10,
         2.02559258e-09, -6.32997710e-10,  3.16498855e-10,  6.96297464e-10,
         6.3